# Initialization

In [ ]:
!pip install ffmpeg-python
!pip install sounddevice

import matplotlib.pyplot as plt
import numpy as np

from scipy import signal
from scipy.signal import chirp

import sounddevice as sd
import matplotlib.pyplot as plt


# Filter Functions

In [ ]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normalCutoff = cutoff / nyq
    b, a = signal.butter(order, normalCutoff, btype='low')
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

# Task 1.1 : Single tone signal processing



In [ ]:
def play_and_record(freq,sample_rate,duration,sd):    
    sd.default.channels   = 1
    sd.default.samplerate = sample_rate
    
    #TODO implement this function (hint: use np.linspace for t)
    t = ...
    x = ...
    y = sd.playrec(...)
    
    return y


In [ ]:
freq         = 10000 # single tone frequency in Hz
sample_rate  = 48000 # sampling rate in samples/s
duration     = 2     # in seconds

rec = play_and_record(freq, sample_rate, duration, sd)

Let us see how the recorded signal looks like in time, frequency and how does the spectrogram look like.

In [ ]:
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
plt.plot(t,rec)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Time domain")

In [ ]:
window_length = rec.shape[0]

rec2 = rec[:,0]

fft = np.abs(np.fft.rfft(rec2)) # Frequency Domain 
freqs = np.multiply(np.fft.rfftfreq(window_length), sample_rate)

plt.plot(freqs,fft)                                     
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.title("Frequency Domain")

In [ ]:
window_length = 500
num_windows = int(np.size(rec2) / window_length)
split_sound = np.array(np.split(rec2,num_windows))

spec = np.abs(np.apply_along_axis(np.fft.rfft, 1, split_sound))
freqs               = np.multiply(np.fft.rfftfreq(window_length), sample_rate)
t =  np.linspace(0, duration, num_windows, endpoint=False)
plt.pcolormesh(t, freqs, np.transpose(spec), shading='gouraud')
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.title("Spectrogram")

If you did everything correctly, you should see a peak in the frequency domain representation and a bright horizontal line (at y = 10,000 Hz) in the spectrogram, you can also change the frequency above and make sure that the plots are changing accordingly

# Task 1.2 FMCW signal processing


In [ ]:
def play_and_record_chirp(freq_low,freq_high,sample_rate,chirp_duration,total_duration,sd):

    sd.default.channels   = 1
    sd.default.samplerate = sample_rate
    
    #TODO implement this function
    t = ...
    chirp_signal = chirp(...)
    num_chirp = total_duration // chirp_duration
    tx = np.tile(chirp_signal, num_chirp)
    rx = sd.playrec(...)

    return tx, rx


In [ ]:
sample_rate = 48000
freq_low    = 17000
freq_high   = 23000

chirp_length   = .05
total_duration = 5   # extend final sound file to "total_duration" seconds

tx, rx = play_and_record_chirp(freq_low,freq_high,sample_rate,chirp_length,total_duration,sd)

#### Now we will segment our signal into small parts where each part will contain atmost 1 FMCW chirp signal. We will focus on one of these segments for now and we will also plot the FFT of this segment (recieved FMCW chirp)

In [ ]:
tx_sig = tx
rx_sig = rx[:,0]

num_chirps_recorded = int(rx_sig.shape[0] // (chirp_length * sample_rate))

# trim audio to a whole number of chirps recorded
rx_sig = rx_sig[:int(num_chirps_recorded * chirp_length * sample_rate)]

rx_data = np.array(np.split(rx_sig, num_chirps_recorded))
tx_data = np.tile(tx_sig[0:int(chirp_length*sample_rate)], (num_chirps_recorded, 1))

time_to_drop = 1
segments_to_drop = int(time_to_drop/chirp_length)
rx_data = rx_data[segments_to_drop:]
tx_data = tx_data[segments_to_drop:]

window_length = rx_data.shape[1]

sample_index = 0

rx_data_sample = rx_data[sample_index]
tx_data_sample = tx_data[sample_index]


fft = np.abs(np.fft.rfft(butter_bandpass_filter(rx_data_sample, freq_low, freq_high, sample_rate)))
freqs = np.multiply(np.fft.rfftfreq(window_length), sample_rate)

plt.plot(freqs, fft)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.title('Frequency Domain - Recieved FMCW signal')

#### Great! Now we can mix this recieved signal with the transmitted signal and we should see a strong peak at a very low frequency, can you think of a reason of why we get this peak?

In [ ]:
multiplied = np.multiply(rx_data_sample, tx_data_sample)
multiplied_fft = np.abs(np.fft.rfft(multiplied))

plt.plot(freqs, multiplied_fft)
plt.xlabel("Frequency")
plt.ylabel("Amplitude")
plt.title("Frequency Domain - Downconverted FMCW signal ")

#### Let us filter everything else (i.e. use a low pass filter) so that we can zoom in on this peak and let us record the peak location, how can we map this peak location to a distance? 

In [ ]:
multiplied_fft = np.abs(np.fft.rfft(butter_lowpass_filter(multiplied, 5000, sample_rate)))
peak_location = np.argmax(multiplied_fft)

plt.plot(freqs, multiplied_fft)
plt.xlabel("Frequency")
plt.ylabel("Amplitute")
plt.title("Frequency Domain - Low pass FMCW signal")

print(peak_location)

#### Now we will repeat the same steps for all of the segments and let us also see how the spectrogram changes over time

In [ ]:
all_multiplied = np.multiply(rx_data, tx_data)

lowpass_cutoff = 5000  

all_multiplied      = np.apply_along_axis(lambda x: butter_lowpass_filter(x, lowpass_cutoff, sample_rate), 1, all_multiplied)
all_multiplied_ffts = np.abs(np.apply_along_axis(np.fft.rfft, 1, all_multiplied))

t_stamp = 0
for s_indx in range(5):
    rec2 = all_multiplied[s_indx]
    split_sound = np.array(np.split(rec2,int(np.size(rec2) / 50)))

    spec = np.abs(np.apply_along_axis(np.fft.rfft, 1, split_sound))
    window_length       = 50
    freqs               = np.multiply(np.fft.rfftfreq(window_length), sample_rate)
    
    t =  np.linspace(t_stamp, t_stamp+0.05, 48, endpoint=False)
    t_stamp = t_stamp + 0.05

    plt.figure(s_indx)
    plt.pcolormesh(t, freqs, np.transpose(spec), shading='gouraud')
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.title("Spectrogram")

# Task 2.1 Background Subtraction


In [ ]:
def background_subtract(all_multiplied_ffts):
    #TODO: For each chirp, subtract the background signal and store them in "after_subtraction" variable
    # The background signal is the mean values of all the chirps.
    background = np.mean(...)
    after_subtraction = ...

    return after_subtraction 


In [ ]:
# play sound
sample_rate = 48000
freq_low    = 17000
freq_high   = 23000
chirp_length   = .05
total_duration = 10
tx, rx = play_and_record_chirp(freq_low,freq_high,sample_rate,chirp_length,total_duration,sd)
rx = rx.squeeze()


In [ ]:
# mix tx and rx to get intermediate signal
num_chirp = int(round(total_duration / chirp_length))
all_multiplied = np.multiply(rx, tx).reshape(num_chirp, -1)
lowpass_cutoff = 5000  
all_multiplied      = np.apply_along_axis(lambda x: butter_lowpass_filter(x, lowpass_cutoff, sample_rate), 1, all_multiplied)
all_multiplied_ffts = np.abs(np.apply_along_axis(np.fft.rfft, 1, all_multiplied))

# background subtraction
subtracted = background_subtract(np.fft.fftshift(all_multiplied_ffts, axes=(1,)))
all_peak_locations = np.apply_along_axis(np.argmax, 1, np.fft.fftshift(all_multiplied_ffts, axes=(1,)))
print(all_peak_locations)


In [ ]:
median_peak_location = int(np.median(all_peak_locations))
peak_window_size     = 100
window_range_start   = median_peak_location - peak_window_size/2
window_range         = np.arange(window_range_start,
                         window_range_start + peak_window_size,
                         dtype=np.int32)

window_length       = rx_data.shape[1]
freqs               = np.multiply(np.fft.rfftfreq(window_length), sample_rate)
subtracted_filtered = subtracted[:, window_range]


t = np.arange(subtracted_filtered.shape[0])*0.05
freqs = freqs - np.max(freqs)/2

plt.pcolormesh(t, freqs[window_range], subtracted_filtered.T, shading='gouraud')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('After background subtraction')

In [ ]:
# Returns the moving average of the input signal
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

# like argmax, but returns the mean of the largest n values
def get_largest_n_mean(array, n):
    return np.mean(np.argpartition(array, -n)[-n:])

In [ ]:
# calculate the peak distance
#print(subtracted_filtered.shape)
#MEAN_WINDOW = 2     # use the largest `MEAN_WINDOW` values instead of a single argmax

argmaxes = np.apply_along_axis(np.argmax, 1, subtracted_filtered)
#mean_argmaxes = np.apply_along_axis(lambda x: get_largest_n_mean(x, MEAN_WINDOW), 1, subtracted_filtered)
plt.plot(t, argmaxes)
plt.xlabel('Time (s)')
plt.ylabel('Peak Locations')
plt.title('Peak Locations')

# Task 2.2 Converting Peak Values to Distance


In [ ]:
def get_median_index(idx):
    # use global values
    return idx + window_range_start - median_peak_location

In [ ]:
def idx_to_distance(idx):
    #TODO implement the equation above
    delta_f = ...
    slope = ...      # Hz/s
    distance = ...   # remember to divide by 2 for the round-trip distance
    
    return distance


In [ ]:
MOVING_AVERAGE_LENGTH = 5
MEDIAN_FILTER_LENGTH  = 7

med_filtered = signal.medfilt(argmaxes, MEDIAN_FILTER_LENGTH)
argmax_distances_med = np.apply_along_axis(get_median_index, 0, med_filtered)
argmax_distances = np.apply_along_axis(idx_to_distance, 0, argmax_distances_med)

distances_to_plot = moving_average(argmax_distances, MOVING_AVERAGE_LENGTH)
timestamps_to_plot = np.arange(len(distances_to_plot)) * chirp_length

plt.plot(timestamps_to_plot,
         distances_to_plot)
plt.xlabel("Time (s)")
plt.ylabel("Distance (m)")
plt.title("Distance variation")


# Task 3.1 Breath Monitoring

Before playing the sound, use your hands to hold the mac laptop so that its speaker is directly facing your chest. Keep the mac about 40 cm away from your chest.

In [ ]:
# play sound
sample_rate = 48000
freq_low    = 17000
freq_high   = 23000
chirp_length   = 0.05
total_duration = 10
tx, rx = play_and_record_chirp(freq_low,freq_high,sample_rate,chirp_length,total_duration,sd)

In [ ]:
# mix the signal
rx_all = rx.squeeze()
sample_in_chirp = int(chirp_length * sample_rate)
mixed = np.multiply(rx_all, tx).reshape(-1, sample_in_chirp)

lowpass_cutoff = 5000 
mixed = np.apply_along_axis(lambda x: butter_lowpass_filter(x, lowpass_cutoff, sample_rate), 1, mixed)

In [ ]:
#TODO: implement the missing code in the block
mixed_sub = ...             # background subtraction of mixed
mixed_sub_fft = np.apply_along_axis(np.fft.rfft, 1, mixed_sub)
mixed_sub_fft_abs = ...     # the amplitude (abs) of the fft result
mixed_sub_fft_phase = ...   # the phase of the fft result

mixed_sub_peak_inds = np.apply_along_axis(np.argmax, 1, mixed_sub_fft_abs)
print('after subtraction peak ind: \n', mixed_sub_peak_inds)
print(f'mean {np.mean(mixed_sub_peak_inds)}, median: {np.median(mixed_sub_peak_inds)}')


In [ ]:
# show the breathing pattern
range_bin = 15
unwrap_phase = np.unwrap(mixed_sub_fft_phase[:, range_bin])

plt.figure()
plt.plot(unwrap_phase)
plt.title('Breath Monitoring')
plt.show()


If you don't see the expected curve, that is because the `range_bin` is not correct. That value depends on the distance between the speaker and your chest. Use the following codes to search for the correct one

In [ ]:
# search for the correct plot
for range_bin in range(10, 20):
    unwrap_phase = np.unwrap(mixed_sub_fft_phase[:, range_bin])

    plt.figure()
    plt.plot(unwrap_phase)
    plt.title(f'Breath Monitoring, raneg_bin={range_bin}')
    plt.show()